In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.path.abspath('../src/'))
sys.path.append(os.path.abspath('../'))

In [2]:
from src.central_ids import IDS
from src.utils import cal_metric, print_results, test_model
from src.dataset import CANDataset
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import numpy as np
from pathlib import Path
from sklearn.metrics import f1_score

# Test cross-domain data

In [ ]:
# data_dir = '../../Data/road/Preprocessed_Data/1/'
ckpt_files = '../save/Kia/lightning_logs/version_1/checkpoints/epoch=49-val_loss=0.00-val_f1=0.9992.ckpt'
model = IDS.load_from_checkpoint(ckpt_files, C=2)

In [14]:
def evaluate(model):
    transform = None
    data_dir = '../../Data/LISA/Federated_Data/Preprocessed_Data/{}/1/'
    car_models = ['Kia', 'Tesla', 'BMW']
    res = []
    total_labels = np.array([])
    total_preds = np.array([])
    for car_model in car_models:
        print(f'Run test on {car_model} dataset')
        test_dataset = CANDataset(root_dir=Path(data_dir.format(car_model))/'test', is_binary=True, transform=transform)
        test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, 
                                pin_memory=True, sampler=None)
        trainer = pl.Trainer(enable_checkpointing=False, logger=False)
        results = trainer.predict(model, dataloaders=test_loader)
        labels = np.concatenate([x['labels'] for x in results])
        preds = np.concatenate([x['preds'] for x in results])
        f1 = f1_score(labels, preds)
        res.append(f1)
        total_labels = np.concatenate([total_labels, labels])
        total_preds = np.concatenate([total_preds, preds])
    car_models.append('Global')
    f1_global = f1_score(total_labels, total_preds)
    res.append(f1_global)
    return (car_models, res)

Run test on Kia dataset


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/htnu/opt/miniconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1789: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
/Users/htnu/opt/miniconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:225: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

Run test on Tesla dataset


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/htnu/opt/miniconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1789: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
/Users/htnu/opt/miniconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:225: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

Run test on BMW dataset


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/htnu/opt/miniconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1789: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
/Users/htnu/opt/miniconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:225: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

In [ ]:
car_models, res = evaluate(model)

In [17]:
for name, value in zip(car_models, res):
    print(name, round(value, 4))

Kia 0.9982
Tesla 0.8535
BMW 0.6564
Global 0.7723


# Test local

## Kia

In [3]:
# data_dir = '../../Data/road/Preprocessed_Data/1/'
ckpt_files = '../save/Kia/lightning_logs/version_7/checkpoints/epoch=19-val_loss=0.03-val_f1=0.9902.ckpt'
model = IDS.load_from_checkpoint(ckpt_files, C=2)

In [7]:
car_model = 'Kia'
data_dir = f'../../Data/LISA/{car_model}/train1000/'
test_model(data_dir, model)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(1.0, 0.0)

In [5]:
car_model = 'Kia'
data_dir = f'../../Data/LISA/{car_model}/test/'
test_model(data_dir, model)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(0.9858085148910654, 0.009040747028862478)

## Tesla

In [8]:
# data_dir = '../../Data/road/Preprocessed_Data/1/'
ckpt_files = '../save/Tesla/lightning_logs/version_4/checkpoints/epoch=19-val_loss=0.07-val_f1=0.9757.ckpt'
model = IDS.load_from_checkpoint(ckpt_files, C=2)

In [9]:
car_model = 'Tesla'
data_dir = f'../../Data/LISA/{car_model}/train1000/'
results = test_model(data_dir, model)
results

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(1.0, 0.0)

In [10]:
car_model = 'Tesla'
data_dir = f'../../Data/LISA/{car_model}/test/'
results = test_model(data_dir, model)
results

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(0.9386993292505103, 0.02641499949733588)

## BMW

In [11]:
# data_dir = '../../Data/road/Preprocessed_Data/1/'
ckpt_files = '../save/BMW/lightning_logs/version_5/checkpoints/epoch=19-val_loss=1.37-val_f1=0.6318.ckpt'
model = IDS.load_from_checkpoint(ckpt_files, C=2)

In [12]:
car_model = 'BMW'
data_dir = f'../../Data/LISA/{car_model}/train1000'
test_model(data_dir, model)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(1.0, 0.0)

In [13]:
car_model = 'BMW'
data_dir = f'../../Data/LISA/{car_model}/test/'
test_model(data_dir, model)

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1767: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=4)`.
  category=PossibleUserWarning,
/home/dhkim1/miniconda3/envs/torch/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 0it [00:00, ?it/s]

(0.7171348314606741, 0.37155243981182545)